In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree

Your task is to modify `KNNClassifier` class from your practice in class. The `KNNClassifier` class with empty methods is provided below. Please, modify it to do all tasks.

In [2]:
class KNNClassifier(object):
    
    def __init__(self, max_dist=1., use_kd_tree=False, use_weights=False):
        """
        This is a constructor of the class. 
        Here you can define parameters (max_dist) of the class and 
        attributes, that are visible within all methods of the class.
        
        Parameters
        ----------
        max_dist : float
            Maximum distance between an object and its neighbors.
        """
        
        # Make this parameter visible in all methods of the class
        self.max_dist = max_dist
        self.use_kd_tree = use_kd_tree
        self.use_weights = use_weights
        
        self.X_train = None
        self.Y_train = None
                
    
    def fit(self, X, y):
        """
        This method trains the KNN classifier. 
        Actualy, the KNN classifier has no training procedure.
        It just remembers data (X, y) that will be used for predictions.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        y : numpy.array, shape = (n_objects)
            1D array with the object labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        ### Your code here
        self.X_train = X
        self.Y_train = y
        
    def predict(self, X):
        """
        This methods performs labels prediction for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted : numpy.array, shape = (n_objects)
            1D array with predicted labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        # Create an empty list for predicted labels
        y_predicted = []
        
        ### Replace this line with your code:
        for main_x in X:
            if self.use_kd_tree:
                kd_tree = cKDTree(data=self.X_train, leafsize=30)
                dist_neighbors = kd_tree.query_ball_point(x=main_x, r=self.max_dist)
                
                if len(dist_neighbors) == 0:
                    dist_neighbors = kd_tree.query(main_x, 1)
           
            else:  
                calc_dist = lambda main_x, x : np.sqrt((main_x[0] - x[0]) ** 2 + (main_x[1] - x[1]) ** 2)
                dists = np.array([calc_dist(main_x, one_x) for one_x in self.X_train])
                
                neighbors_values = dists[dists <= self.max_dist]
                neighbors_indexes = np.argsort(dists)
                dist_neighbors = neighbors_indexes[:len(neighbors_values)]
                
                if len(dist_neighbors) == 0:
                    dist_neighbors = neighbors_indexes[0]
                
            neighbors_labels = self.Y_train[dist_neighbors]
            
            if self.use_weights:
                calc_weights = lambda main_x, x : 1.0 / np.sqrt((main_x[0] - x[0]) ** 2 + (main_x[1] - x[1]) ** 2)
                weights = np.array([calc_weights(main_x, one_x) for one_x in self.X_train])
                
                first_class, second_class = [], []
                for node in dist_neighbors:
                    if self.Y_train[node] == 0:
                        first_class.append(node)
                    elif self.Y_train[node] == 1:
                        second_class.append(node)
                    
                first_weights, second_weights = 0, 0
                for index in first_class:
                    first_weights += weights[index]
                
                for index in second_class:
                    second_weights += weights[index]
                    
                if first_weights < second_weights:
                    y_predicted.append(1)
                else:
                    y_predicted.append(0)
                
            else:
                unique_labels, label_counts = np.unique(neighbors_labels, return_counts=True)
                label_max_count = unique_labels[label_counts == label_counts.max()][0]
                y_predicted.append(label_max_count)
        
        ### The end of your code
            
        return np.array(y_predicted) # return numpy.array
    
    
    def predict_proba(self, X):
        """
        This methods performs prediction of probabilities of each class for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted_proba : numpy.array, shape = (n_objects, n_classes)
            2D array with predicted probabilities of each class. 
            Example:
                y_predicted_proba = [[0.1, 0.9],
                                     [0.8, 0.2], 
                                     [0.0, 1.0], 
                                     ...]
        """
        
        # Create an empty list for predictions
        y_predicted_proba = []
        
        ### Replace these lines with your code:
        for main_x in X:
            kd_tree = cKDTree(data=self.X_train, leafsize=30)
            dist_neighbors = kd_tree.query_ball_point(x=main_x, r=self.max_dist)

            if len(dist_neighbors) == 0:
                dist_neighbors = kd_tree.query(main_x, 1)
            
            #neighbors_labels = self.Y_train[dist_neighbors]
                    
            probabilities = [0, 0]
            calc_weights = lambda main_x, x : 1.0 / np.sqrt((main_x[0] - x[0]) ** 2 + (main_x[1] - x[1]) ** 2)
            weights = np.array([calc_weights(main_x, one_x) for one_x in self.X_train])
            
            for index in dist_neighbors:
                if self.Y_train[index] == 0:
                    probabilities[0] += weights[index]
                else:
                     probabilities[1] += weights[index]
            
            probabilities /= sum(probabilities)
            y_predicted_proba.append(probabilities)
        
        ### The end of your code
            
        return np.array(y_predicted_proba) # return numpy.array

### Task 1 (1 point) <br/>
Create a matrix of object features `X` and vector of labels `y` for N=1000 objects using `sklearn.datasets.make_moons()` function from scikit-learn library. Also, set up random state in the function `random_state=42` and `noise=0.2`. To open the function description use `Shift` + `Tab` .

In [3]:
### Your code here
X, y = make_moons(n_samples=1000, random_state=42, noise=0.2)


### Check your solution
ans = np.array([[-0.112,  0.52 ],
                [ 1.143, -0.343]])
assert np.array_equal(np.round(X[:2], 3), ans), ('Check your solution.')

### Task 2 (1 point) <br/>

Split the sample into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library. Use `random_state = 42` and `test_size = 0.5`.

In [4]:
### Your code here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


### Check your solution
ans = np.array([[ 0.77 , -0.289],
                [ 0.239,  1.041]])
assert np.array_equal(np.round(X_train[:2], 3), ans), ('Check your solution.')

### Task 3 (2 points) <br/>

Modify class `KNNClassifier` above and implement `predict()` method that uses **max_dist** parameter to select neighbors like it's shown in the second figure (radius search). If there is no any object within **max_dist**, make decision based on the closest neighbor.

<img src="img/knn2.png" width="600">

In [5]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)


### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 781 ms, sys: 97 µs, total: 781 ms
Wall time: 781 ms
Test accuracy of KNN classifier:  0.964


### Task 4 (2 points) <br/>

There are an algorithm [kd-tree](https://en.wikipedia.org/wiki/K-d_tree) that helps to find neighbors faster. Using [scipy.spatial.cKDTree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.cKDTree.html#scipy.spatial.cKDTree) function modify you classifier to speed up **predict** method. Use `leafsize=30` in `KDTree`. Similar to `max_dist` option, add option `use_kd_tree = True/False` to your classifier.

In [6]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)

### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 125 ms, sys: 3.81 ms, total: 129 ms
Wall time: 128 ms
Test accuracy of KNN classifier:  0.964


### Task 5 (3 points) <br/>

Now modify the **predict** method to provide prediction with neighbors weights.

<img src="img/wv1.png">

<img src="img/wv2.png">

We propose you to use the following weights:

$$
w_{i} = \frac{1}{\rho(x, x_{i})}
$$

Similar to `max_dist` option, add option `use_weights = True/False` to your classifier.

In [7]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.968, ('Check your solution.')

CPU times: user 913 ms, sys: 56 µs, total: 913 ms
Wall time: 912 ms
Test accuracy of KNN classifier:  0.968


### Task 6 (3 points) <br/>

Develop **predict_proba** method of the classifier. For each object this method returns probability that the object belongs to each of the classes. 

For each object $x$ probability for each class is defined as:

$$
p_{c}(x) = \frac{g_{c}(x)}{\sum_{i=1}^{C} g_{i}(x)}
$$

where $C$ is number of classes.

Then, the object has a vector of probabilities:

$$
p(x) = (p_{1}(x), p_{2}(x), ..., p_{C}(x))
$$

Use neighbors weights as in Task 5.

In [8]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict_proba = knn.predict_proba(X_test) # measure time for prediction

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows



### Check your solution
ans = np.array([[0.046, 0.954],
                [0.962, 0.038]])
assert np.array_equal(np.round(y_test_predict_proba[:2], 3), ans), ('Check your solution.')

CPU times: user 910 ms, sys: 3.81 ms, total: 913 ms
Wall time: 914 ms
